# Setup the dataframe

In [1]:
import pandas as pd
import os
from datetime import datetime, timezone
import time
import matplotlib.pyplot as plt

all_data = pd.DataFrame()

# Be careful with this. If there is more than one CSV file only the last one will be used. 
for file in os.listdir('./'):
    if file.endswith('csv'):
        all_data = pd.read_csv('./' + file, sep=';')
        print("Using file: " + file)


Using file: 2020_08_02_15_45_54.csv


## Setup the Date_Time_(GMT) column

In [2]:
start = time.time()

all_data['PLC_TIME(Timedate48)'] = all_data['PLC_TIME(Timedate48)'].astype(str)

def getUnixTime(date):
    return date.split(',')[0]

def convertToUTC(date):
    return datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

def convertToGMT(unix_timestamp):
    utcTime = datetime.fromtimestamp(unix_timestamp, timezone.utc)
    localTime = utcTime.astimezone()
    return localTime.strftime("%Y-%m-%d %H:%M:%S")
    
all_data['Date_Time_(GMT)'] = all_data['PLC_TIME(Timedate48)'].apply(lambda x: getUnixTime(x))
all_data['Date_Time_(GMT)'] = all_data['Date_Time_(GMT)'].astype('int32')
all_data['Date_Time_(GMT)'] = all_data['Date_Time_(GMT)'].apply(lambda x: convertToGMT(x))
column_name = 'Date_Time_(GMT)'
first_column = all_data.pop(column_name)
all_data.insert(0, column_name, first_column)
all_data['Date_Time_(GMT)'] = pd.to_datetime(all_data['Date_Time_(GMT)'])
all_data = all_data.sort_values(by='Date_Time_(GMT)')

duration = time.time() - start
print("Total time: %10.2f" % duration)

all_data.tail()

Total time:       2.31


,Date_Time_(GMT),TIME,PLC_TIME_OFFSET(Integer8),PLC_TIME_CV(Enum2),PLC_TIME(Timedate48),EVR_VehicleNumber(Unsigned16),EVR_TIME_CV(Enum2),EVR_TIME(Timedate48),EVR_Speed(Unsigned16),EVR_Speed_CV(Enum2),...,AL1213_COACH7(Boolean1),AL1213_COACH8(Boolean1),AL1213_COACH9(Boolean1),AL1213_COACH10(Boolean1),AL1213_COACH11(Boolean1),AL1213_COACH12(Boolean1),AL1213_COACH13(Boolean1),AL1213_COACH14(Boolean1),AL1213_COACH15(Boolean1),AL1213_COACH16(Boolean1)
130756,2020-08-03 10:21:43,"66947,072",0,1,"1596446503,000",15004,1,"1596446503,000",0,1,...,0,0,0,0,0,0,0,0,0,0
130757,2020-08-03 10:21:43,"66947,584",0,1,"1596446503,000",15004,1,"1596446504,000",0,1,...,0,0,0,0,0,0,0,0,0,0
130759,2020-08-03 10:21:44,"66948,608",0,1,"1596446504,000",15004,1,"1596446505,000",0,1,...,0,0,0,0,0,0,0,0,0,0
130758,2020-08-03 10:21:44,"66948,096",0,1,"1596446504,000",15004,1,"1596446504,000",0,1,...,0,0,0,0,0,0,0,0,0,0
130760,2020-08-03 10:21:45,"66949,120",0,1,"1596446505,000",15004,1,"1596446505,000",0,1,...,0,0,0,0,0,0,0,0,0,0


#### Cleanup the NaN entries

In [3]:
print("Rows before: " + str(all_data.shape[0]))
all_data = all_data.dropna(how='all')
print("Rows after: " + str(all_data.shape[0]))
all_data.head()

Rows before: 130761
Rows after: 130761


,Date_Time_(GMT),TIME,PLC_TIME_OFFSET(Integer8),PLC_TIME_CV(Enum2),PLC_TIME(Timedate48),EVR_VehicleNumber(Unsigned16),EVR_TIME_CV(Enum2),EVR_TIME(Timedate48),EVR_Speed(Unsigned16),EVR_Speed_CV(Enum2),...,AL1213_COACH7(Boolean1),AL1213_COACH8(Boolean1),AL1213_COACH9(Boolean1),AL1213_COACH10(Boolean1),AL1213_COACH11(Boolean1),AL1213_COACH12(Boolean1),AL1213_COACH13(Boolean1),AL1213_COACH14(Boolean1),AL1213_COACH15(Boolean1),AL1213_COACH16(Boolean1)
0,2020-08-02 15:45:59,"0,000",0,1,"1596379559,000",15004,1,"1596379559,000",0,1,...,0,0,0,0,0,0,0,0,0,0
1,2020-08-02 15:45:59,"0,512",0,1,"1596379559,000",15004,1,"1596379560,000",0,1,...,0,0,0,0,0,0,0,0,0,0
2,2020-08-02 15:46:00,"1,024",0,1,"1596379560,000",15004,1,"1596379560,000",0,1,...,0,0,0,0,0,0,0,0,0,0
3,2020-08-02 15:46:00,"1,536",0,1,"1596379560,000",15004,1,"1596379561,000",0,1,...,0,0,0,0,0,0,0,0,0,0
4,2020-08-02 15:46:01,"2,048",0,1,"1596379561,000",15004,1,"1596379561,000",0,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
HABD_columns = [col for col in all_data if 'Temp' in col and not 'Alarm' in col]
HABD_columns.pop(0) # removes useless column 115, 115 ...
print(len(HABD_columns))

273


In [5]:
HABD_columns.insert(0, 'Date_Time_(GMT)')
print(len(HABD_columns))
print(HABD_columns[0])

274
Date_Time_(GMT)


### Add the HABD Temperatures to filtered_data data frame

In [6]:
filtered_data = pd.DataFrame()
for column_name in HABD_columns:
    filtered_data[column_name] = all_data[column_name]

In [7]:
print("Rows: " + str(filtered_data.shape[0]))
filtered_data.tail()

Rows: 130761


,Date_Time_(GMT),HABD_TempDiff(Boolean1),HABD_TempDiff_Car9(Bitset8),HABD_TempDiff_Car8(Bitset8),HABD_TempDiff_Car7(Bitset8),HABD_TempDiff_Car6(Bitset8),HABD_TempDiff_Car5(Bitset8),HABD_TempDiff_Car4(Bitset8),HABD_TempDiff_Car3(Bitset8),HABD_TempDiff_Car2(Bitset8),...,HABD_TempCh11_Car2(Integer16),HABD_TempCh11_Car1(Integer16),HABD_TempCh10_Car9(Integer16),HABD_TempCh10_Car8(Integer16),HABD_TempCh10_Car7(Integer16),HABD_TempCh10_Car6(Integer16),HABD_TempCh10_Car5(Integer16),HABD_TempCh10_Car4(Integer16),HABD_TempCh10_Car3(Integer16),HABD_TempCh10_Car2(Integer16)
130756,2020-08-03 10:21:43,0,0,0,0,0,0,0,0,0,...,24,25,0,24,24,25,25,23,25,24
130757,2020-08-03 10:21:43,0,0,0,0,0,0,0,0,0,...,24,25,0,24,24,25,25,23,25,24
130759,2020-08-03 10:21:44,0,0,0,0,0,0,0,0,0,...,24,25,0,24,24,25,25,23,25,24
130758,2020-08-03 10:21:44,0,0,0,0,0,0,0,0,0,...,24,25,0,24,24,25,25,23,25,24
130760,2020-08-03 10:21:45,0,0,0,0,0,0,0,0,0,...,24,25,0,24,24,25,25,23,25,24


In [14]:
df = pd.DataFrame()
dftest = pd.DataFrame()
df['Date_Time_(GMT)'] = all_data['Date_Time_(GMT)']

for col in HABD_columns[1:]:
    df[col] = all_data[col]
    print(df[col].value_counts())

0    130761
Name: HABD_TempDiff(Boolean1), dtype: int64
0    130761
Name: HABD_TempDiff_Car9(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car8(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car7(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car6(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car5(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car4(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car3(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car2(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car16(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car15(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car14(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car13(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car12(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car11(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car10(Bitset8), dtype: int64
0    130761
Name: HABD_TempDiff_Car1(

In [9]:
for i in HABD_columns[1:]:
    print(i)

HABD_TempDiff(Boolean1)
HABD_TempDiff_Car9(Bitset8)
HABD_TempDiff_Car8(Bitset8)
HABD_TempDiff_Car7(Bitset8)
HABD_TempDiff_Car6(Bitset8)
HABD_TempDiff_Car5(Bitset8)
HABD_TempDiff_Car4(Bitset8)
HABD_TempDiff_Car3(Bitset8)
HABD_TempDiff_Car2(Bitset8)
HABD_TempDiff_Car16(Bitset8)
HABD_TempDiff_Car15(Bitset8)
HABD_TempDiff_Car14(Bitset8)
HABD_TempDiff_Car13(Bitset8)
HABD_TempDiff_Car12(Bitset8)
HABD_TempDiff_Car11(Bitset8)
HABD_TempDiff_Car10(Bitset8)
HABD_TempDiff_Car1(Bitset8)
HABD_TempCh10_Car1(Integer16)
HABD_TempCh16_Car16(Integer16)
HABD_TempCh16_Car15(Integer16)
HABD_TempCh16_Car14(Integer16)
HABD_TempCh16_Car13(Integer16)
HABD_TempCh16_Car12(Integer16)
HABD_TempCh16_Car11(Integer16)
HABD_TempCh16_Car10(Integer16)
HABD_TempCh15_Car16(Integer16)
HABD_TempCh15_Car15(Integer16)
HABD_TempCh15_Car14(Integer16)
HABD_TempCh15_Car13(Integer16)
HABD_TempCh15_Car12(Integer16)
HABD_TempCh15_Car11(Integer16)
HABD_TempCh15_Car10(Integer16)
HABD_TempCh14_Car16(Integer16)
HABD_TempCh14_Car15(Intege

### Filter out the low temperatures